<a href="https://colab.research.google.com/github/hakprj/2D_3D/blob/main/gaussian_splatting_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/trianglesplatting/triangle-splatting --recursive

Cloning into 'triangle-splatting'...
remote: Enumerating objects: 122, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 122 (delta 21), reused 9 (delta 8), pack-reused 89 (from 1)
Receiving objects: 100% (122/122), 1.20 MiB | 10.48 MiB/s, done.
Resolving deltas: 100% (35/35), done.
Submodule 'submodules/diff-triangle-rasterization' (https://github.com/trianglesplatting/diff-triangle-rasterization.git) registered for path 'submodules/diff-triangle-rasterization'
Submodule 'submodules/simple-knn' (https://gitlab.inria.fr/bkerbl/simple-knn.git) registered for path 'submodules/simple-knn'
Cloning into '/content/triangle-splatting/submodules/diff-triangle-rasterization'...
remote: Enumerating objects: 31, done.        
remote: Counting objects: 100% (31/31), done.        
remote: Compressing objects: 100% (21/21), done.        
remote: Total 31 (delta 8), reused 31 (delta 8), pack-reused 0 (from 0)        
Receiving objects: 

In [4]:
!pip -q install --upgrade pip
!pip -q install torch==2.4.0 torchvision==0.19.0 --index-url https://download.pytorch.org/whl/cu124

!pip -q install tqdm plyfile open3d lpips mediapy opencv-python
%cd /content/triangle-splatting/submodules/simple-knn
!pip install . --no-build-isolation
%cd /content/triangle-splatting/submodules/diff-triangle-rasterization
!pip install . --no-build-isolation




In [6]:
%cd /content/triangle-splatting/submodules/diff-triangle-rasterization
!pip install . --no-build-isolation


/content/triangle-splatting/submodules/diff-triangle-rasterization
Processing /content/triangle-splatting/submodules/diff-triangle-rasterization
  Preparing metadata (pyproject.toml) ... done
  Created wheel for diff_triangle_rasterization: filename=diff_triangle_rasterization-0.0.0-cp312-cp312-linux_x86_64.whl size=3363522 sha256=a8daafea124931c1572c16088b8e68dd4b2e83d9ae91950fa4dec2062d66dca9
  Stored in directory: /root/.cache/pip/wheels/19/c5/67/075fd6c47289d5046e540af2173ca940c6f1cb9a44ae3927a5
Successfully built diff_triangle_rasterization


In [8]:
import zipfile
!rm -rf /content/data/
!mkdir -p /content/data
zip_path = "/content/GP_360_Large_Top_Masked.zip"
out_dir = "/content/data/"

with zipfile.ZipFile(zip_path, 'r') as z:
    z.extractall(out_dir)


In [ ]:
%cd /content/triangle-splatting/
!python train.py -s /content/data/GP_360_Large_Top_Masked -m /content/GP_360_Large_Top_Masked --resolution 1 --iterations 15000

/content/triangle-splatting
2025-12-26 00:58:32.182972: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-26 00:58:32.200788: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766710712.222214   34127 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766710712.228809   34127 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1766710712.245357   34127 computation_placer.cc:177] computation placer already registered. Please chec

In [18]:
%cd /content/triangle-splatting/

!python render.py -m /content/GP_360_Large_Top_Masked

Looking for config file in /content/GP_360_Large_Top_Masked/cfg_args
Config file found: /content/GP_360_Large_Top_Masked/cfg_args
Rendering /content/GP_360_Large_Top_Masked
Loading trained model at iteration 7000 [25/12 23:21:51]
Reading camera 101/101 [25/12 23:21:52]
Loading Training Cameras [25/12 23:21:52]
Loading Test Cameras [25/12 23:21:58]
/content/triangle-splatting/scene/triangle_model.py:255: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explici

In [19]:
%cd /content/triangle-splatting/

!python create_video.py -m /content/GP_360_Large_Top_Masked

Looking for config file in /content/GP_360_Large_Top_Masked/cfg_args
Config file found: /content/GP_360_Large_Top_Masked/cfg_args
Creating video for /content/GP_360_Large_Top_Masked
Loading trained model at iteration 7000
Reading camera 101/101
Loading Training Cameras
Loading Test Cameras
/content/triangle-splatting/scene/triangle_model.py:255: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_sa

In [35]:
%cd /content/triangle-splatting/

!python train_game_engine.py \
  -s /content/data/GP_360_Large_Top_Masked \
  -m /content/output/GP_360_engine \
  -i images \
  -r 2  \
  --iterations 15000


/content/triangle-splatting
2025-12-26 00:15:29.548849: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-26 00:15:29.566317: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766708129.587345   23374 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766708129.593854   23374 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1766708129.609954   23374 computation_placer.cc:177] computation placer already registered. Please chec

In [37]:
%cd /content/triangle-splatting/

!python create_off.py \
  --checkpoint_path /content/output/GP_360_engine/point_cloud/iteration_15000/point_cloud_state_dict.pt \
  --output_name mesh_colored5_15K.off


/content/triangle-splatting
saved mesh_colored5_15K.off


In [ ]:
from google.colab import drive
drive.mount('/content/drive')